In [15]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx

In [35]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [43]:
# text = '''
# [ 종목 투자등급 ] 
# 당사는 개별 종목에 대해 향후 1 년간 +15% 이상의 절대수익률이 기대되는 종목에 대해 Buy(매수) 의견을 제시합니다. 또한 절대수
# 익률 -15~+15%가 예상되는 종목에 대해 Hold(보유) 의견을, -15% 이하가 예상되는 종목에 대해 Sell(매도) 의견을 제시합니다. 밸류
# 에이션 방법 등 절대수익률 산정은 개별 종목을 커버하는 애널리스트의 추정에 따르며, 목표주가 산정이나 투자의견 변경 주기는 종
# 목별로 다릅니다. 
# [ 산업 투자의견 ] 
# 당사는 산업에 대해 향후 1 년간 해당 업종의 수익률이 과거 수익률에 비해 양호한 흐름을 보일 것으로 예상되는 경우에 Positive(긍정
# 적) 의견을 제시하고 있습니다. 또한 향후 1 년간 수익률이 과거 수익률과 유사한 흐름을 보일 것으로 예상되는 경우에 Neutral(중립적) 
# 의견을, 과거 수익률보다 부진한 흐름을 보일 것으로 예상되는 경우에 Negative(부정적) 의견을 제시하고 있습니다. 산업별 수익률 전

# 망은 해당 산업 내 분석대상 종목들에 대한 담당 애널리스트의 분석과 판단에 따릅니다. 
# [ 당사 조사분석자료의 투자등급 부여 비중 ] (기준일: 2021년 12 월 31 일) 
# 투자등급 매수 중립 매도 합계 
# 금융투자상품의 비중 97.7% 2.3% 0.0% 100.0% 
# '''
text = """
투자의견 매수, 목표주가 100,000원 유지.
2022년 EPS 3.331원에 Target PER 29.3배 적용.
Target PER은 글로벌 게임 peer의 2021년 PER 평균 대비 20% 할증 적용.
동사의 P2E/NFT 게임 사업에 대한 기대감 반영하여 20% 프리미엄 적용.
연간 매출 1조원 달성. 
오딘의 매출 안정화와 신사업의 성장세 4분기 매출액 2,867억원, 영업이익 479억원기록. 
컨센서스 하회하는 실적 기록. 
자회사 라이온하트스튜디오의 연결 편입 과정에서 공정가치평가를 통한 일시적 수익 발생으로 당기순이익 4,573억원 기록.
모바일 게임 매출 2,000억원 기록. 
가장 비중이 큰 오딘 매출의 하향 안정화 영향. 오딘의 평균 일매출 17억원 수준으로 추정.
대만 출시를 위해, 2/10부터 양대 마켓에서의 사전 예약 시작과 본격적인
마케팅 진행 예정. 오딘의 2022년 한국과 대만의 일 평균 매출은 각각 15억, 7억원으로 추정. PC 게임 매출 193억원 기록.
기타 매출에 반영되는 카카오VX는 매 분기 최고 실적 기록. 
골프에 대한 관심이 커지면서 동사의 골프 연습장 사업 역시 지속적으로 성장할 것으로 기대. 
이외에 세나테크놀로지가 4분기부터 연결 실적에 편입되면서 매출 성장에 기여. 
4분기 매출 증가에는 기여했으나, 반도체 수급 이슈에 따라 해당 사업부의 매출원가 증가로 이익은 감소. 
2022년부터는 반도체 수급 이슈가 해소됨에 따라 이익 개선 기대.
P2E 방향성은 확인. 다만, 신중하고 천천히
2/8 진행했던 보라 2.0 브랜딩 행사에서 2Q22부터 출시될 10여 종의 게임 장르 공개. 
가장 기대가 컸던 오딘의 P2E 도입은 공개하지 않았으며, 회사는 대만 출시를 통한 오딘의 글로벌 IP로서의 성장에 집중할 것으로 발표.
회사의 향후 성장성을 위해 P2E 게임을 기반으로 토큰 이코노미 및 블록체인 생태계 구축의 방향성은 확인. 
단, 현재 P2E 라인업 공개에 집중하기 보다, 여러 파트너사들과 고도화된 토큰 이코노미 구축을 위한 협력 단계. 
단기적으로 P2E 라인업을 구체적으로 공개하지 않은 것이 회사의 전반적인 사업 기조를 바꾸지는 않을 것.
"""

sentences = sent_tokenize(text)

In [44]:
sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
stop_words = '''
아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 
의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 
하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 
만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 
않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 
틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 
여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 
일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 
할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 
훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 
그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 
바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 
응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 
까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 
하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 
하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 
아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 
혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 
인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 
비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 
통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 
그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 
해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 
했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 
습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 
갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 
요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 
할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 
기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 
전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 
이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 
한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 
하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓
'''
sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]

In [45]:
w2v=Word2Vec(sentence_tokens,size=1,min_count=1,iter=1000)
sentence_embeddings=[[w2v[word][0] for word in words] for words in sentence_tokens]
max_len=max([len(tokens) for tokens in sentence_tokens])
sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [46]:
similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
for i,row_embedding in enumerate(sentence_embeddings):
    for j,column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)

In [47]:
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)

In [48]:
top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])

In [49]:
for sent in sentences:
    if sent in top.keys():
        print(sent)

동사의 P2E/NFT 게임 사업에 대한 기대감 반영하여 20% 프리미엄 적용.
오딘의 2022년 한국과 대만의 일 평균 매출은 각각 15억, 7억원으로 추정.
기타 매출에 반영되는 카카오VX는 매 분기 최고 실적 기록.
회사의 향후 성장성을 위해 P2E 게임을 기반으로 토큰 이코노미 및 블록체인 생태계 구축의 방향성은 확인.


In [54]:
def pdf_summarize(text):
  sentences = sent_tokenize(text)

  sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
  stop_words = '''
  아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 
  하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 
  설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 
  여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 
  할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 
  그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 
  응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 
  하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 
  아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 
  인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 
  통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 
  해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 
  습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 
  요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 
  기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 
  이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 
  하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓
  '''
  sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]

  w2v=Word2Vec(sentence_tokens,size=1,min_count=1,iter=1000)
  sentence_embeddings=[[w2v[word][0] for word in words] for words in sentence_tokens]
  max_len=max([len(tokens) for tokens in sentence_tokens])
  sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]

  similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
  for i,row_embedding in enumerate(sentence_embeddings):  
    for j,column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)

  nx_graph = nx.from_numpy_array(similarity_matrix)
  scores = nx.pagerank(nx_graph)

  top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
  top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])

  for sent in sentences:
    if sent in top.keys():
        sent += sent+"\n"
  
  return sent

In [55]:
text1 = '''
[ 종목 투자등급 ] 
당사는 개별 종목에 대해 향후 1 년간 +15% 이상의 절대수익률이 기대되는 종목에 대해 Buy(매수) 의견을 제시합니다. 또한 절대수
익률 -15~+15%가 예상되는 종목에 대해 Hold(보유) 의견을, -15% 이하가 예상되는 종목에 대해 Sell(매도) 의견을 제시합니다. 밸류
에이션 방법 등 절대수익률 산정은 개별 종목을 커버하는 애널리스트의 추정에 따르며, 목표주가 산정이나 투자의견 변경 주기는 종
목별로 다릅니다. 
[ 산업 투자의견 ] 
당사는 산업에 대해 향후 1 년간 해당 업종의 수익률이 과거 수익률에 비해 양호한 흐름을 보일 것으로 예상되는 경우에 Positive(긍정
적) 의견을 제시하고 있습니다. 또한 향후 1 년간 수익률이 과거 수익률과 유사한 흐름을 보일 것으로 예상되는 경우에 Neutral(중립적) 
의견을, 과거 수익률보다 부진한 흐름을 보일 것으로 예상되는 경우에 Negative(부정적) 의견을 제시하고 있습니다. 산업별 수익률 전

망은 해당 산업 내 분석대상 종목들에 대한 담당 애널리스트의 분석과 판단에 따릅니다. 
[ 당사 조사분석자료의 투자등급 부여 비중 ] (기준일: 2021년 12 월 31 일) 
투자등급 매수 중립 매도 합계 
금융투자상품의 비중 97.7% 2.3% 0.0% 100.0% 
'''
text2 = """
투자의견 매수, 목표주가 100,000원 유지.
2022년 EPS 3.331원에 Target PER 29.3배 적용.
Target PER은 글로벌 게임 peer의 2021년 PER 평균 대비 20% 할증 적용.
동사의 P2E/NFT 게임 사업에 대한 기대감 반영하여 20% 프리미엄 적용.
연간 매출 1조원 달성. 
오딘의 매출 안정화와 신사업의 성장세 4분기 매출액 2,867억원, 영업이익 479억원기록. 
컨센서스 하회하는 실적 기록. 
자회사 라이온하트스튜디오의 연결 편입 과정에서 공정가치평가를 통한 일시적 수익 발생으로 당기순이익 4,573억원 기록.
모바일 게임 매출 2,000억원 기록. 
가장 비중이 큰 오딘 매출의 하향 안정화 영향. 오딘의 평균 일매출 17억원 수준으로 추정.
대만 출시를 위해, 2/10부터 양대 마켓에서의 사전 예약 시작과 본격적인
마케팅 진행 예정. 오딘의 2022년 한국과 대만의 일 평균 매출은 각각 15억, 7억원으로 추정. PC 게임 매출 193억원 기록.
기타 매출에 반영되는 카카오VX는 매 분기 최고 실적 기록. 
골프에 대한 관심이 커지면서 동사의 골프 연습장 사업 역시 지속적으로 성장할 것으로 기대. 
이외에 세나테크놀로지가 4분기부터 연결 실적에 편입되면서 매출 성장에 기여. 
4분기 매출 증가에는 기여했으나, 반도체 수급 이슈에 따라 해당 사업부의 매출원가 증가로 이익은 감소. 
2022년부터는 반도체 수급 이슈가 해소됨에 따라 이익 개선 기대.
P2E 방향성은 확인. 다만, 신중하고 천천히
2/8 진행했던 보라 2.0 브랜딩 행사에서 2Q22부터 출시될 10여 종의 게임 장르 공개. 
가장 기대가 컸던 오딘의 P2E 도입은 공개하지 않았으며, 회사는 대만 출시를 통한 오딘의 글로벌 IP로서의 성장에 집중할 것으로 발표.
회사의 향후 성장성을 위해 P2E 게임을 기반으로 토큰 이코노미 및 블록체인 생태계 구축의 방향성은 확인. 
단, 현재 P2E 라인업 공개에 집중하기 보다, 여러 파트너사들과 고도화된 토큰 이코노미 구축을 위한 협력 단계. 
단기적으로 P2E 라인업을 구체적으로 공개하지 않은 것이 회사의 전반적인 사업 기조를 바꾸지는 않을 것.
"""

In [56]:
pdf_summarize(text1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


'[ 당사 조사분석자료의 투자등급 부여 비중 ] (기준일: 2021년 12 월 31 일) \n투자등급 매수 중립 매도 합계 \n금융투자상품의 비중 97.7% 2.3% 0.0% 100.0%[ 당사 조사분석자료의 투자등급 부여 비중 ] (기준일: 2021년 12 월 31 일) \n투자등급 매수 중립 매도 합계 \n금융투자상품의 비중 97.7% 2.3% 0.0% 100.0%\n'

In [57]:
pdf_summarize(text2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


'단기적으로 P2E 라인업을 구체적으로 공개하지 않은 것이 회사의 전반적인 사업 기조를 바꾸지는 않을 것.'